In [ ]:
!pip install emoji
!pip install preprocessor

In [ ]:
import pandas as pd
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import emoji
import plotly.express as px
from collections import defaultdict
import nltk
from collections import Counter
import preprocessor as p
import codecs

In [ ]:
nltk.download('nps_chat')
nltk.download('punkt')

[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
myfile = open('chat.txt')
content = myfile.read()
chat = defaultdict(list)
content = content.split('\n')
for line in content:
  try:
    person = line.split(':')[2][7:]
    text = nltk.sent_tokenize(line.split(':')[3:][0])
    chat[person].extend(text)
  except:
    pass

In [ ]:
def dateCheck(s):
  pattern = '\d{4}-\d{2}-\d{2}, (\d{2}|\d{1}):\d{2}:\d{2} (AM|PM|am|pm)'
  result = re.search(pattern, s)
  if result:
    return result
  return False

In [ ]:
dateCheck('[2021-02-21, 10:59:44 PM] Krishna: Hi! Nice!')

<regex.Match object; span=(1, 24), match='2021-02-21, 10:59:44 PM'>

In [ ]:
def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime[:dateTime.find(',')][1:],dateTime[dateTime.find(','):][1:]
    date =re.sub('\D', ' ', date).strip()
    date = date.replace(' ', '-') 
    time, author = time[:time.find(']')],time[time.find(']'):][1:]
    author, message = author[:author.find(':')],author[author.find(':'):][1:]
    return date, time.strip(), author.strip(), message.strip()

In [ ]:
data = []
conversation = 'chat.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    parsedData = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if dateCheck(line): 
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear()
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [ ]:
posts = nltk.corpus.nps_chat.xml_posts()

def extract_features(post):
  features={}
  for word in nltk.word_tokenize(post):
    features['contains({})'.format(word.lower())]=True
  return features

fposts = [(extract_features(p.text),p.get('class')) for p in posts]
test_size = int(len(fposts)*0.1)
train_set,test_set = fposts[test_size:],fposts[:test_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.classify(extract_features('What is your score'))

'whQuestion'

In [ ]:
#def emoji_used(str1):
  #return ''.join(t for t in str1 if t in emoji.UNICODE_EMOJI)

print(len(chat))
for key, val in chat.items():
  emojis = []
  for text in val:
    token = nltk.word_tokenize(text)
    for temp in token:
      if(temp in emoji.UNICODE_EMOJI_ENGLISH):
        emojis.append(temp)
  count = Counter(emojis).most_common()[:10]
  print('{}s emojis:\n{}\n'.format(key,emojis))
  print('Common: {}\n\n'.format(count))

2
Harsh Patel Computer Sciences emojis:
['👍', '👍', '👍', '👍', '👍🏻']

Common: [('👍', 4), ('👍🏻', 1)]


Krishnas emojis:
['🙄', '🤣', '✌🏻', '😅', '👍🏻']

Common: [('🙄', 1), ('🤣', 1), ('✌🏻', 1), ('😅', 1), ('👍🏻', 1)]




In [ ]:
df = pd.DataFrame(parsedData, columns=['Date','Time','Author','Message'])
df.tail()
#df.to_csv('test_data.csv')

,Date,Time,Author,Message
1077,2021-05-04,11:46:14 AM,Harsh Patel Computer Science,I mean project or some research
1078,2021-05-04,11:47:18 AM,Krishna,Research itself
1079,2021-05-04,11:50:43 AM,Harsh Patel Computer Science,Ok
1080,2021-05-16,8:08:25 PM,Krishna,CS_802_Programming_Assignment.pdf • ‎1 page ‎d...
1081,2021-05-16,8:08:51 PM,Harsh Patel Computer Science,Thanks bro


In [ ]:
df["Author"].replace({"Harsh Patel Computer Science": "Person1", "Krishna": "Person2"}, inplace=True)
df.tail()

,Date,Time,Author,Message
1077,2021-05-04,11:46:14 AM,Person1,I mean project or some research
1078,2021-05-04,11:47:18 AM,Person2,Research itself
1079,2021-05-04,11:50:43 AM,Person1,Ok
1080,2021-05-16,8:08:25 PM,Person2,CS_802_Programming_Assignment.pdf • ‎1 page ‎d...
1081,2021-05-16,8:08:51 PM,Person1,Thanks bro


In [ ]:
#Credits: Poojan Vyas
def remove_sc(message):
    message = codecs.decode(message, 'unicode_escape').lower()
    message = emoji.demojize(message.encode('utf-16', 'surrogatepass').decode('utf-16'))
    message = message.replace("’", "'")
    message = re.sub(r"&amp;", " ", message) # Remove '&amp;'
    message = re.sub(r"\\n", "", message) # Remove '\\n'
    message = re.sub("[^a-z0-9 ]", " ", message) # Remove special characters
    message = re.sub(" +", " ", message) # Remove extra spaces and convert into lowercase
    return message

In [ ]:
for i,v in enumerate(df['Message']):
    message = v.replace('@', ' ')
    #message = p.clean(message) # Remove http links
    message = remove_sc(message)
    df.loc[i, "Message"] = message.strip() # Strip spaces from the begining and the end of the sentence

In [ ]:
df.head()

,Date,Time,Author,Message
0,2021-02-21,10:59:44 PM,Person1,hi krishna
1,2021-02-22,1:50:19 AM,Person2,hi nice
2,2021-02-22,2:09:09 AM,Person1,do you have any idea in question 4 d what we h...
3,2021-02-22,2:10:08 AM,Person1,i mean we have to do one attitude reduct or no...
4,2021-02-22,2:12:49 AM,Person2,i am starting my assignment now


In [ ]:
msg_data = df.loc[:, "Message"]
#print(msg_data)

In [ ]:
training_txt = ''.join(msg_data[:round(len(msg_data)*0.6)])
val_txt = ''.join(msg_data[round(len(msg_data)*0.6):round(len(msg_data)*0.8)])
test_txt = ''.join(msg_data[round(len(msg_data)*0.8):])

with open('info.train', 'w', encoding='utf-8') as file:
    file.write(training_txt)
    
with open('info.valid', 'w', encoding='utf-8') as file:
    file.write(val_txt)
        
with open('info.test', 'w', encoding='utf-8') as file:
    file.write(test_txt)

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 6.5 MB/s 
  Using cached pybind11-2.7.0-py2.py3-none-any.whl (199 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3092698 sha256=3c56f28c46a25dad36db4d023d63ae6a91998c885b0cc045ae227494b6318f56
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext

In [ ]:
model = fasttext.train_unsupervised(input="info.train")

In [ ]:
model.test("info.test")

(0, nan, nan)